# Boyish Scraping

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from random import randint
import time, urllib, json, unicodedata

In [2]:
# Headers to imitate browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Mobile Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.5',
    'DNT': '1',
    'Referer': 'https://www.google.co.uk',
    'Upgrade-Insecure-Requests': '1'
}

In [3]:
main_URL = 'https://www.boyish.com'
URL_extension_for_products = '/products.json?limit=250&page='
current_page_number = 1


all_URLs_and_JSON = {}
current_product_list = None


while current_product_list != []:
    # make the full json url
    url = main_URL + URL_extension_for_products + str(current_page_number)
    print(url)
    
    # Get the page
    req = urllib.request.Request(url, headers=headers)
    with urllib.request.urlopen(req) as request:
        s = request.read()
        response = json.loads(s)
        current_product_list = response['products']
    
    time.sleep(randint(4,6))
    
    # construct the URL
    for product in current_product_list:
        product_url = main_URL + '/products/' + product['handle']
        all_URLs_and_JSON[product_url] = product
        
    current_page_number += 1

https://www.boyish.com/products.json?limit=250&page=1
https://www.boyish.com/products.json?limit=250&page=2


In [4]:
print(len(all_URLs_and_JSON))
print(all_URLs_and_JSON)

174
{'https://www.boyish.com/products/the-tommy-krush-groove': {'id': 5970122899623, 'title': 'The Tommy | Krush Groove', 'handle': 'the-tommy-krush-groove', 'body_html': '<p>Straight up versatile. The Tommy is a classic high waisted rigid straight jean in a dark indigo wash that you\'ll keep in your closet forever. These upcycled jeans include GRS certified recycled cotton and REFIBRA™ Technology, reducing its carbon footprint and reducing water usage.</p>\n<p>REFIBRA™ Technology takes fashion industry scraps and recycles the waste, which is then blended in with TENCEL™, a natural raw material from eucalyptus trees that is responsibly produced to minimize environmental impact. This zero waste jean will be your new favorite second skin.</p>\n<p>Front Rise: 10 3/4"<br>Inseam: 27"<br>Leg Opening: 14 1/2"<br>Button Fly</p>', 'published_at': '2021-03-01T16:01:43-08:00', 'created_at': '2020-10-22T12:58:46-07:00', 'updated_at': '2021-03-01T16:17:15-08:00', 'vendor': 'Boyish Jeans', 'product_

In [5]:
def find_product_name(soup, product_JSON):
    
    product_title = product_JSON['title'].strip()
    
    return product_title

In [6]:
def find_material(soup, product_JSON):
    materials = {}
    
    materials_container = soup.find('div', class_ = 'Product__Tabs')
    
    materials_string = materials_container.find('button', class_ = 'Collapsible__Button Heading u-h6').text.strip()
    
    materials_string = materials_string[14:]
    
    material_list_cleaned = []
    
    if '/' in materials_string:
        material_list_cleaned = materials_string.split('/')

    else:
        the_materials_split = materials_string.split(' ')
        current_material = ""
        how_many_digits = 0

        for index,item in enumerate(the_materials_split):

            if item[0].isdigit() and how_many_digits == 0:
                current_material = current_material + ' ' + item
                how_many_digits += 1

            elif item[0].isdigit() and how_many_digits == 1:
                material_list_cleaned.append(current_material.strip())
                current_material = ""
                current_material = current_material + ' ' + item

            else:
                current_material = current_material + ' ' + item
                if index == len(the_materials_split) - 1:
                    material_list_cleaned.append(current_material.strip())
        
        
    materials_list = [item.strip().replace('%', '% &ND') for item in material_list_cleaned]

    materials_split = [item.split('&ND') for item in materials_list]

    for item in materials_split:
        materials[item[1].strip()] = item[0].strip()

    return materials

In [7]:
def find_color(soup, product_JSON):
    
    color_container = product_JSON['options'][0]
    
    if color_container['name'] == 'Color':
        color = color_container['values'][0].lower()
    else:
        color = 'not defined'
    
    return [color]

In [8]:
def find_price(soup, product_JSON):

    price_text = soup.find('span', class_ = 'ProductMeta__Price Price Text--subdued u-h4').text.strip()
        
    if price_text[0] == '$':
        price_text = price_text[1:] 
    
    price = ('USD', price_text)
    
    return price

In [9]:
def find_image(soup, product_JSON):
    
    image_url = product_JSON['images'][0]['src']

    return image_url

In [10]:
def find_brand(soup, product_JSON):
    
    brand = product_JSON['vendor']
    
    return brand

In [11]:
def find_description(soup, product_JSON):
    
    description_container = soup.find('div', class_ = 'Rte')
    
    if description_container.text:
        description = [description_container.text.strip()]
    else:
        description_container.find_all('p')
    
        description = []

        for item in description_container:
            if item.find('br'):
                replaced_string = item.text.replace('\"', ' inches\"')
                split_lines = replaced_string.split('\"')
                for line in split_lines:
                    description.append(line.replace('\n', ' ').strip())
            else:
                description.append(item.text.replace('\n', ' ').strip())
    
    return description

In [12]:
def find_category(soup, product_JSON):
    
    category = [product_JSON['product_type'].strip().lower()]
    
    return category

In [13]:
def scrape_single_product(URL, html_soup, product_JSON):
    product = {}
    
#     print('got name')
    product['Name'] = find_product_name(html_soup, product_JSON)
    
#     print('getting material')
    product['Material'] = find_material(html_soup, product_JSON)
    
# #     print('getting color')
    product['Color'] = find_color(html_soup, product_JSON)
        
# #     print('getting price')
    product['Price'] = find_price(html_soup, product_JSON)
    
# #     print('getting URL')
    product['URL'] = URL

# #     print('getting image')
    product['Image'] = find_image(html_soup, product_JSON)
    
# #     print('getting brand name')
    product['Brand_name'] = find_brand(html_soup, product_JSON)
    
# #     print('getting description')
    product['Description'] = find_description(html_soup, product_JSON)
    
# #     print('getting category')
    product['Category'] = find_category(html_soup, product_JSON)
    
    return product

In [14]:
def is_available(product_JSON):
    variants = product_JSON['variants']
    
    is_available = False
    
    for variant in variants:
        availability = variant['available']
        if availability == True:
            is_available = True
    
    return is_available

In [15]:
# These 3 cells scrape a single product
# URL_to_scrape = 'https://www.boyish.com/products/the-julian-like-crazy'
# page = requests.get(URL_to_scrape, headers = headers)
# single_soup = bs(page.content, 'html.parser')

In [16]:
# product_as_json = None
# URL_to_scrape_JSON = URL_to_scrape + ".json"

# single_req = urllib.request.Request(URL_to_scrape_JSON, headers=headers)
# with urllib.request.urlopen(single_req) as request:
#     s = request.read()
#     response = json.loads(s)
#     product_as_json = response['product']
#     print(product_as_json)

In [17]:
# singleProductResult  = [scrape_single_product(URL_to_scrape, single_soup, product_as_json)]

# print(singleProductResult)

# product_data_frame = pd.DataFrame(data = singleProductResult,)

# product_data_frame.to_csv('single_item.csv', encoding='utf-8-sig')

# pd.DataFrame(data = singleProductResult)

In [18]:
all_products = []
failed_urls = []
unavailable_products = []

# counter = 0

for url, the_JSON in all_URLs_and_JSON.items():
    try:

        print(url)
        
        if is_available(the_JSON):
            print(is_available(the_JSON))
            page = requests.get(url, headers = headers)
            soup = bs(page.content, 'html.parser')

            product = scrape_single_product(url, soup, the_JSON)
            print(product)
            all_products.append(product)
            
        else:
            unavailable_products.append(url)

        time.sleep(randint(5,15))
  
    except:
        failed_urls.append(url)
    
#     if counter == 15:
#         break
#     counter += 1

print('\nSummary\n')
print(len(failed_urls), 'Failed URLs: ', failed_urls)
print(len(unavailable_products), 'Unavailable Products: ', unavailable_products)
print('Total products collected: ', len(all_products))
print('Done!')

https://www.boyish.com/products/the-tommy-krush-groove
True
{'Name': 'The Tommy | Krush Groove', 'Material': {'OCS Certified Organic Cotton': '60%', 'Tencel ™ Lyocell': '40%'}, 'Color': ['krush groove'], 'Price': ('USD', '168'), 'URL': 'https://www.boyish.com/products/the-tommy-krush-groove', 'Image': 'https://cdn.shopify.com/s/files/1/0229/4949/4856/products/TheTommy-KrushGroove-Front.jpg?v=1603488516', 'Brand_name': 'Boyish Jeans', 'Description': ['Straight up versatile. The Tommy is a classic high waisted rigid straight jean in a dark indigo wash that you\'ll keep in your closet forever. These upcycled jeans include GRS certified recycled cotton and REFIBRA™ Technology, reducing its carbon footprint and reducing water usage.\nREFIBRA™ Technology takes fashion industry scraps and recycles the waste, which is then blended in with TENCEL™, a natural raw material from eucalyptus trees that is responsibly produced to minimize environmental impact. This zero waste jean will be your new fa

https://www.boyish.com/products/the-tommy-top-five
True
{'Name': 'The Tommy | Top Five', 'Material': {'OCS Certified Organic Cotton': '17%', 'GRS Certified Recycled Cotton': '43%', 'Refibra ™ X Tencel ™ Lyocell': '40%'}, 'Color': ['top five'], 'Price': ('USD', '168'), 'URL': 'https://www.boyish.com/products/the-tommy-top-five', 'Image': 'https://cdn.shopify.com/s/files/1/0229/4949/4856/products/TheTommy-TopFive-Front.jpg?v=1614734198', 'Brand_name': 'Boyish Jeans', 'Description': ['Classic straight leg. The Tommy in Top Five is a high waisted rigid straight jean in a medium indigo with subtle destroy and a raw hem. These upcycled jeans include GRS certified recycled cotton and REFIBRA™ Technology, reducing its carbon footprint and reducing water usage. REFIBRA™ Technology takes fashion industry scraps and recycles the waste, which is then blended in with TENCEL™, a natural raw material from eucalyptus trees that is responsibly produced to minimize environmental impact. This zero waste 

https://www.boyish.com/products/the-georgy-white
True
https://www.boyish.com/products/the-jimmy-black
True
{'Name': 'The Jimmy | Black', 'Material': {'OCS Certified Organic Cotton': '74%', 'Refibra™ X Tencel ™ Lyocell': '26%'}, 'Color': ['black'], 'Price': ('USD', '88'), 'URL': 'https://www.boyish.com/products/the-jimmy-black', 'Image': 'https://cdn.shopify.com/s/files/1/0229/4949/4856/products/Boyish5faeea147614075faeea1476260.682514885faeea1476260.jpg?v=1611282649', 'Brand_name': 'Boyish Jeans', 'Description': ['Versatile vintage-inspired. The Jimmy in black is a super soft terry sweatshirt with roomy sleeves and a cinched waist. Reminiscent of the 90s, this timeless staple includes OCS Certified organic cotton and REFIBRA™ Technology, reducing its carbon footprint and reducing water usage.\nREFIBRA™ Technology takes fashion industry scraps and recycles the waste, which is then blended in with TENCEL™, a natural raw material from eucalyptus trees that is responsibly produced to minim

https://www.boyish.com/products/the-jeffery-blue-velvet
True
{'Name': 'The Jeffery | Blue Velvet', 'Material': {'Organic Cotton': '70%', 'GRS Certified Recycled Cotton': '30%'}, 'Color': ['blue velvet'], 'Price': ('USD', '188'), 'URL': 'https://www.boyish.com/products/the-jeffery-blue-velvet', 'Image': 'https://cdn.shopify.com/s/files/1/0229/4949/4856/products/Boyish5faeead933c3715faeead933d3d.369010705faeead933d3d.jpg?v=1610671334', 'Brand_name': 'Boyish Jeans', 'Description': ['Ready for an adventure. Inspired by safari vibes, the Jeffery is a rigid jacket with utility pockets and a tie-belt. This jacket includes recycled cotton reducing its carbon footprint and reducing water.\nFront HPS: 28 3/4"\xa0Sweep: 43 1/4"'], 'Category': ['jackets']}
https://www.boyish.com/products/the-maxwell-steel-magnolia
https://www.boyish.com/products/the-maxwell-the-crowd
True
{'Name': 'The Maxwell | The Crowd', 'Material': {'OCS Certified Organic Cotton': '75%', 'Tencel ™ Lyocell': '25%'}, 'Color': ['

https://www.boyish.com/products/the-kirby-bullit
https://www.boyish.com/products/the-charley-the-navigator
True
{'Name': 'The Charley | The Navigator', 'Material': {'Organic Cotton': '70%', 'GRS Certified Recycled Cotton': '30%'}, 'Color': ['the navigator'], 'Price': ('USD', '188'), 'URL': 'https://www.boyish.com/products/the-charley-the-navigator', 'Image': 'https://cdn.shopify.com/s/files/1/0229/4949/4856/products/TheCharley-Navigator-Front.jpg?v=1591977397', 'Brand_name': 'Boyish Jeans', 'Description': ['Back in time. Here is our take on rigid high waisted wide leg jeans. The Charley comes in a medium light wash with a raw hem and exposed button fly. This jean includes recycled cotton reducing its carbon footprint and reducing water usage.\nFront Rise: 13 1/2"Inseam: 31 3/4"Leg Opening: 21 3/4"Button Fly'], 'Category': ['jeans']}
https://www.boyish.com/products/the-charley-trapeze
True
{'Name': 'The Charley | Trapeze', 'Material': {'Organic Cotton': '70%', 'GRS Certified Recycled Co

...

https://www.boyish.com/products/the-billy-toxic-avenger
True
{'Name': 'The Billy | Toxic Avenger', 'Material': {'OCS Certified Organic Cotton': '18%', 'GRS Certified Recycled Cotton': '41%', 'Tencel ™ Lyocell': '41%'}, 'Color': ['toxic avenger'], 'Price': ('USD', '168'), 'URL': 'https://www.boyish.com/products/the-billy-toxic-avenger', 'Image': 'https://cdn.shopify.com/s/files/1/0229/4949/4856/products/TheBilly-ToxicAvenger-Front.jpg?v=1599788406', 'Brand_name': 'Boyish Jeans', 'Description': ['Your new classic. The Billy is a high waisted rigid skinny jean reminiscent of the most perfect hip hugging vintage denim in a vintage marbled black eco wash. These upcycled jeans include organic cotton that is OCS certified, and Tencel™ Lyocell, a natural raw material from eucalyptus trees that is responsibly produced to minimize environmental impact, reducing its carbon footprint and water usage.\nFront Rise: 11 1/4"Inseam: 29"Leg Opening: 11 1/2"Button Fly'], 'Category': ['jeans']}
https://ww

https://www.boyish.com/products/the-harvey-the-crowd
True
{'Name': 'The Harvey | The Crowd', 'Material': {'Organic Cotton': '70%', 'GRS Certified Recycled Cotton': '30%'}, 'Color': ['the crowd'], 'Price': ('USD', '188'), 'URL': 'https://www.boyish.com/products/the-harvey-the-crowd', 'Image': 'https://cdn.shopify.com/s/files/1/0229/4949/4856/products/TheHarvey-TheCrowd-Front-Close.jpg?v=1581365077', 'Brand_name': 'Boyish Jeans', 'Description': ['Born to be wild. The Harvey is inspired by your favorite vintage denim jacket. It is rigid and cropped with a raw hem and comes in a vintage khaki wash. This jacket includes recycled cotton reducing its carbon footprint and reducing water.\nVintage Khaki Wash\nFront High Point Shoulder: 20 1/2"\xa0Sweep: 41"\n** Sale items are non-refundable // Exchange or store credit only **'], 'Category': ['jackets']}
https://www.boyish.com/products/the-brady-river-of-no-return
True
{'Name': 'The Brady | River Of No Return', 'Material': {'Cotton': '70%', 'Rec

https://www.boyish.com/products/the-billy-krush-groove
True
{'Name': 'The Billy | Krush Groove', 'Material': {'OCS Certified Organic Cotton': '60%', 'Tencel ™ Lyocell': '40%'}, 'Color': ['krush groove'], 'Price': ('USD', '168'), 'URL': 'https://www.boyish.com/products/the-billy-krush-groove', 'Image': 'https://cdn.shopify.com/s/files/1/0229/4949/4856/products/THE-BILLY_KRUSH-GROOVE_F2_2x_tiny.jpg?v=1606809977', 'Brand_name': 'Boyish Jeans', 'Description': ['Your new classic. The Billy is a high waisted rigid skinny jean reminiscent of the most perfect hip hugging vintage denim in a perfect dark eco wash. These upcycled jeans include organic cotton that is OCS certified, and Tencel™ Lyocell, a natural raw material from eucalyptus trees that is responsibly produced to minimize environmental impact, reducing its carbon footprint and water usage.\nFront Rise: 11 1/4"Inseam: 29"Leg Opening: 11 1/2"Button Fly'], 'Category': ['jeans']}
https://www.boyish.com/products/the-mason-secret-garden
T

https://www.boyish.com/products/the-dusty-sunrise
True
{'Name': 'The Dusty | Sunrise', 'Material': {'GOTS Certified Organic Cotton': '100%'}, 'Color': ['sunrise'], 'Price': ('USD', '138'), 'URL': 'https://www.boyish.com/products/the-dusty-sunrise', 'Image': 'https://cdn.shopify.com/s/files/1/0229/4949/4856/products/TheDusty-Sunrise-Front.jpg?v=1599245626', 'Brand_name': 'Boyish Jeans', 'Description': [''], 'Category': ['']}
https://www.boyish.com/products/the-robby-vertigo
True
{'Name': 'The Robby | Vertigo', 'Material': {'OCS Certified Organic Cotton': '60%', 'Tencel ™ Lyocell': '40%'}, 'Color': ['vertigo'], 'Price': ('USD', '78'), 'URL': 'https://www.boyish.com/products/the-robby-vertigo', 'Image': 'https://cdn.shopify.com/s/files/1/0229/4949/4856/products/Robby-Vertigo-Front-Close.jpg?v=1596136590', 'Brand_name': 'Boyish Jeans', 'Description': [''], 'Category': ['shirts']}
https://www.boyish.com/products/the-cody-accatone
True
{'Name': 'The Cody | Accatone', 'Material': {'OCS Certif

https://www.boyish.com/products/the-jagger-white
True
{'Name': 'The Jagger | White', 'Material': {'OCS Certified Organic Cotton': '60%', 'Refibra™ X Tencel ™ Lyocell': '40%'}, 'Color': ['white'], 'Price': ('USD', '48'), 'URL': 'https://www.boyish.com/products/the-jagger-white', 'Image': 'https://cdn.shopify.com/s/files/1/0229/4949/4856/products/TheJagger-White-Front.jpg?v=1587510127', 'Brand_name': 'Boyish Jeans', 'Description': ['An elevated basic. The Jagger is a super soft jersey slim crew neck tee with short sleeves that looks great with any of our jeans. It\'s a classic staple made with OCS Certified Organic Cotton and Refibra™ Technology, making it an eco-friendly addition to your wardrobe. REFIBRA™ Technology takes fashion industry scraps and recycles the waste, which is then blended in with TENCEL™. This zero waste tee will be your new favorite second skin.\nFront HPS: 22 1/2"Sweep: 33 1/2"'], 'Category': ['shirts']}
https://www.boyish.com/products/the-westley-pandoras-box
True

In [19]:
# fail_again_URLs = []
# for url in failed_urls:
#     try:

#         req = urllib.request.Request(url, headers=headers)
        
#         with urllib.request.urlopen(req) as request:
#             page = request.read()
#             soup = bs(page, 'html.parser')
#             all_products.append(scrape_single_product(url, soup))
            
#             time.sleep(randint(5,15))
            
#     except:
#         fail_again_URLs.append(url)

In [20]:
product_data_frame = pd.DataFrame(data = all_products)

product_data_frame.to_csv('boyish_table.csv', encoding='utf-8-sig')

pd.DataFrame(data = all_products)

,Name,Material,Color,Price,URL,Image,Brand_name,Description,Category
0,The Tommy | Krush Groove,"{'OCS Certified Organic Cotton': '60%', 'Tence...",[krush groove],"(USD, 168)",https://www.boyish.com/products/the-tommy-krus...,https://cdn.shopify.com/s/files/1/0229/4949/48...,Boyish Jeans,[Straight up versatile. The Tommy is a classic...,[jeans]
1,The Casey | Unfaithfully Yours,"{'OCS Certified Organic Cotton': '17%', 'GRS C...",[unfaithfully yours],"(USD, 168)",https://www.boyish.com/products/the-casey-unfa...,https://cdn.shopify.com/s/files/1/0229/4949/48...,Boyish Jeans,[Perfectly imperfect. The Casey is a twist on ...,[jeans]
2,The Julian | Like Crazy,"{'Organic Cotton': '70%', 'GRS Certified Recyc...",[like crazy],"(USD, 248)",https://www.boyish.com/products/the-julian-lik...,https://cdn.shopify.com/s/files/1/0229/4949/48...,Boyish Jeans,[Suit up in Boyish. The versatile Julian blaze...,[jackets]
3,The Liam | Like Crazy,"{'Organic Cotton': '70%', 'GRS Certified Recyc...",[like crazy],"(USD, 188)",https://www.boyish.com/products/the-liam-like-...,https://cdn.shopify.com/s/files/1/0229/4949/48...,Boyish Jeans,[Suit up in Boyish. The Liam in Like Crazy is ...,[pants]
4,The Leon | Like Crazy,"{'Organic Cotton': '70%', 'GRS Certified Recyc...",[like crazy],"(USD, 108)",https://www.boyish.com/products/the-leon-like-...,https://cdn.shopify.com/s/files/1/0229/4949/48...,Boyish Jeans,[From work to play and anywhere in between. Th...,[shorts]
...,...,...,...,...,...,...,...,...,...
146,The Dempsey | La Strada,"{'Cotton': '70%', 'Recycled Cotton': '28%', 'E...",[la strada],"(USD, 158)",https://www.boyish.com/products/the-dempsey-la...,https://cdn.shopify.com/s/files/1/0229/4949/48...,Boyish Jeans,[],[jeans]
147,The Dempsey | Claire's Knee,"{'Cotton': '70%', 'Recycled Cotton': '28%', 'E...",[claire's knee],"(USD, 158)",https://www.boyish.com/products/the-dempsey-cl...,https://cdn.shopify.com/s/files/1/0229/4949/48...,Boyish Jeans,[90's classic. The Dempsey is a high waisted c...,[jeans]
148,The Charley | Vertigo,"{'Organic Cotton': '70%', 'GRS Certified Recyc...",[vertigo],"(USD, 188)",https://www.boyish.com/products/the-charley-ve...,https://cdn.shopify.com/s/files/1/0229/4949/48...,Boyish Jeans,[Back in time. Here is our take on rigid high ...,[jeans]
149,The Mikey | Two For The Road,"{'Cotton': '70%', 'Recycled Cotton 1%': '29%'}",[two for the road],"(USD, 158)",https://www.boyish.com/products/the-mikey-two-...,https://cdn.shopify.com/s/files/1/0229/4949/48...,Boyish Jeans,[High-Rise Comfort Stretch Wide Leg],[jeans]
